# Huggging Face with langchain with OpenSource Models

In [6]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import retrieval_qa

from dotenv import load_dotenv

load_dotenv()
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")


## Read the PDFs from the folder

In [2]:
loader = PyPDFDirectoryLoader("./us-census")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Poverty in States and Metropolitan  \nAreas: 2022\nAmerican Community Survey Briefs\nDecember 2023ACSBR-016By Craig Benson\nINTRODUCTION\nPlanners, policymakers, and community stakeholders \nuse poverty estimates as key indicators to evaluate trends and current economic conditions within com-munities and to make comparisons across demo-graphic groups. Federal and state governments often \nuse these estimates to allocate funds to local com-\nmunities. Government agencies, researchers, and local organizations regularly use these estimates to identify the number of individuals and families eligible for vari-ous programs and to measure economic well-being.\nThis brief uses the 2021 and 2022 American \nCommunity Survey (ACS) 1-year estimates and the \n2021 and 2022 Puerto Rico Community Surveys', metadata={'source': 'us-census/acsbr-016.pdf', 'page': 0})

In [3]:
print("Total number of final documents: ", len(final_documents))

Total number of final documents:  316


## Embeddings Techniques from HugginFace

In [9]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
        model_kwargs={'device' : 'cpu'}
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
import numpy as np

embeddings = np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))
embeddings

array([-1.00630093e-02, -3.98545079e-02, -2.14797799e-02,  5.86761162e-02,
        8.02909508e-02,  5.45422174e-02, -1.89204793e-02, -3.84315941e-03,
       -5.06585911e-02, -9.22430120e-03,  5.20436689e-02,  2.59992275e-02,
       -1.59694161e-02, -1.32009741e-02, -3.79712917e-02, -2.48674769e-02,
       -4.57807072e-02,  5.27276956e-02,  2.56091282e-02,  1.23642311e-02,
        5.47024049e-02, -9.55425110e-03, -7.26953521e-02, -1.91431195e-02,
        5.93195856e-02, -3.60356383e-02, -1.51843517e-04, -5.13967797e-02,
       -4.05045189e-02, -1.59561887e-01,  6.18914468e-03,  2.72260350e-03,
        4.80102375e-02, -6.29634736e-03, -3.37924645e-03, -4.26316299e-02,
        3.80311012e-02,  5.58205582e-02,  8.53930600e-03,  5.33878915e-02,
       -4.03700193e-04, -2.85844021e-02, -1.75249502e-02, -3.33271138e-02,
       -6.52636811e-02, -6.25040308e-02, -1.69884115e-02, -3.36707011e-03,
       -5.80814369e-02, -1.87588464e-02,  3.09326369e-02, -1.79333128e-02,
        1.11762658e-02,  

In [16]:
# View the shape of the embeddings array
print(f"Shape of the Embeddings array: {embeddings.shape}")

Shape of the Embeddings array: (384,)


## Vector Store Creation

In [19]:
## Vector Store Creation
vectorstore = FAISS.from_documents(final_documents[:120], huggingface_embeddings)

In [21]:
# Query using Similarity Search
query = "WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_documents = vectorstore.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


## create Retriever 

In [22]:
retreiver = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k" : 3})
print(retreiver)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7c7aaa5700d0> search_kwargs={'k': 3}


## Create the model

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint

hf=HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

/home/amulyaprasanth/Documents/tutorials/Langchain Course Krishanik/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
